In [1]:
!pip install transformers datasets torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
import torch, transformers, datasets, torchmetrics
from torch.utils.data import DataLoader
from transformers import AutoModelForUniversalSegmentation
from datasets import load_dataset
from transformers import AutoImageProcessor
from torchvision.transforms import PILToTensor
from huggingface_hub import hf_hub_download
import torchvision.transforms as tvt
from torchmetrics.classification import Accuracy, JaccardIndex, Dice, MulticlassCalibrationError
from tqdm.auto import tqdm
import statistics

from copy import deepcopy
import numpy as np
from pathlib import Path
import json
import matplotlib.pyplot as plt

In [3]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('HF_TOKEN'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
model_ckpt = "BhavanaMalla/Mask2Former_Finetuning_cityscapes_swin_railSem19_50epchs_1ptch_1024"
trained_model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/75.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/190M [00:00<?, ?B/s]

In [5]:
trained_model

Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value

In [11]:
for module in trained_model.modules():
    if module.__class__.__name__.startswith('Drop'):
        print(module)

Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=0.0, inplace=False)
Dropout(p=

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !cp /content/drive/MyDrive/Colab\ Notebooks/Master\ Thesis/mask2former_model .

In [ ]:
# model_ckpt = torch.load("./mask2former_model", map_location=torch.device('cpu'))
# model_state_dict = torch.load("/nfs1/malla/Thesis_copy/checkpoints/model.pt")
# model_ckpt.keys()
#trained_model.load_state_dict(torch.load("path"))

In [4]:
def get_modified_labels():
    modified_id2label = {
        0: "road", 1: "sidewalk", 2: "construction_fence",
        3: "rail_raised_rail_embedded", 4: "pole_traffic_light_traffic_sign",
        5: "sky", 6: "human", 7: "tram_track_rail_track", 8: "car_truck",
        9: "on_rails", 10: "vegetation", 11: "trackbed",
        12: "background_terrain"
    }
    modified_label2id = {
        label: id for id, label in modified_id2label.items()
    }
    modified_labels = [label for label in modified_id2label.values()]
    return modified_id2label, modified_label2id, modified_labels

def get_labels():
    data_directory = Path(".")
    json_file_path = data_directory / "labels_info.json"
    if json_file_path.is_file():
        print(f"[INFO]: Found {json_file_path}.Skipping Download...")
    else:
        print("[INFO]: Downloading labels_info.json from hub")
        json_file_path = hf_hub_download(
            repo_id="BhavanaMalla/railsem19-semantic-expanded",
            filename="labels_info.json",
            repo_type="dataset",
            local_dir=data_directory
        )
    with open(json_file_path, "r") as f:
        labels_info = json.load(f)
    id2label = labels_info["id2label"]
    label2id = labels_info["label2id"]
    labels = labels_info["labels"]
    color_palette = labels_info["color_palette"]

    # add the background label
    id2label["19"] = "background"
    label2id["background"] = 19
    labels.append("background")
    color_palette.append([0, 0, 0])

    # correcting the labels
    id2label = {int(key): value.replace('-', '_') \
                for key, value in id2label.items()}
    label2id = {key.replace('-', '_'): value for key, value in label2id.items()}
    labels = [label.replace('-', '_') for label in labels]
    return {"id2label": id2label, "label2id": label2id, "labels": labels,
            "color_palette": color_palette}

class RemapBackground():
    def __call__(self, mask):
        return torch.where(mask > 18, 19, mask)

class RemapLabels():
    def __init__(self):
        self.class_mapping = get_labels()["id2label"]
        self.class_coding = {v: k for k, v in self.class_mapping.items()}

        # Remapping of original 20 labels to 13 labels
        self.modified_labels = {
            0: ["road"], 1: ["sidewalk"],
            2: ["construction", "fence"],
            3: ["rail_raised", "rail_embedded"],
            4: ["pole", "traffic_light", "traffic_sign"],
            5: ["sky"], 6: ["human"],
            7: ["tram_track", "rail_track"],
            8: ["car", "truck"], 9: ["on_rails"],
            10: ["vegetation"], 11: ["trackbed"],
            12: ["background", "terrain"]
        }
        self.modified_ids = {}
        for k, v in self.modified_labels.items():
            self.modified_ids[k] = [self.class_coding[label] for label in v]

    def __call__(self, mask):
        final_label = np.zeros_like(mask)
        for key, val in self.modified_ids.items():
            specific_label = np.zeros_like(mask)
            specific_label = np.where(np.isin(mask, np.array(val)), 1, 0)
            specific_label *= key
            final_label = np.add(final_label, specific_label)
        return torch.from_numpy(final_label)

def get_transforms():
    image_transforms = tvt.Compose([tvt.PILToTensor(),])
    mask_transforms = tvt.Compose([tvt.PILToTensor(),  # Doesnt scale, Converts a PIL Image (H x W x C) to a Tensor of shape (C x H x W). doesnot scale hence using this for masks
                                  RemapBackground(),
                                  RemapLabels(),])
    return image_transforms, mask_transforms

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        semantic_mask = self.dataset[idx]["semantic_mask_label"]
        image_name = self.dataset[idx]["img_Name"]
        if self.transforms:
            image_transforms, mask_transforms = self.transforms
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        else:
            image_transforms, mask_transforms = get_transforms()
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        return {
            "image": transformed_image, "mask": transformed_mask,
            "name": image_name
        }

def _collate_fn(batch):
    images = tuple([sample["image"] for sample in batch])
    segmentation_maps = tuple([sample["mask"].squeeze() for sample in batch])
    batch = processor(images, segmentation_maps=segmentation_maps, return_tensors="pt")
    batch["original_images"] = images
    batch["original_segmentation_maps"] = segmentation_maps
    return batch

In [ ]:
# Plottings
import albumentations as A
import cv2

def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

def visualize_multiple(nrows, ncols, img, transform):
    fig, axes = plt.subplots(nrows,ncols)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    num_iter = 0
    for row in range(nrows):
        for col in range(ncols):
            augmented_img = transform[num_iter](image=img)['image']
            axes[row, col].imshow(augmented_img)
            axes[row, col].grid(False)
            axes[row, col].set_xticks([])
            axes[row, col].set_yticks([])
            num_iter += 1
    return fig, axes

def plot_predictions(images, ground_truth_mask, predicted_mask, uncertainty_map, titles=None):
    id2label, _, _ = get_modified_labels()
    if len(id2label) == 13:
        palette = [
            [128, 64, 128], [244, 35, 232], [189, 122, 36], [0, 254, 254],
             [153, 153, 153], [70, 130, 180], [220, 20, 60], [230, 150, 140],
              [0, 0, 142], [0, 80, 100], [107, 142, 35], [90, 40, 40], [152, 251, 152]
        ]
    else:
        palette = [
            [128, 64, 128], [244, 35, 232], [70, 70, 70], [192, 0, 128],
            [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0],
            [107, 142, 35], [152, 251, 152], [70, 130, 180], [220, 20, 60],
            [230, 150, 140], [0, 0, 142], [0, 0, 70], [90, 40, 40], [0, 80, 100],
            [0, 254, 254], [0, 68, 63]
        ]

    palette = np.array(palette)
    num_images = len(images)
    cols = 4

    fig, axes = plt.subplots(num_images, cols, figsize=(15, 5*num_images))
    axes = np.atleast_2d(axes)

    for i, (img, gt_mask, pred_mask, uncertain_map) in enumerate(zip(images, ground_truth_mask, predicted_mask, uncertainty_map)):
        img_transposed = img.squeeze().permute(1, 2, 0) #
        gt_mask = gt_mask.squeeze()
        pred_mask = pred_mask.squeeze()
        uncertain_map = uncertain_map.squeeze()
        color_seg = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8) # height, width, 3
        color_seg_gt = np.zeros((gt_mask.shape[0], gt_mask.shape[1], 3), dtype=np.uint8)
        for idx, color in enumerate(palette):
            color_seg[pred_mask == idx, :] = color
            color_seg_gt[gt_mask == idx, :] = color
        # Convert to BGR
        color_seg = color_seg[..., ::-1]
        color_seg_gt = color_seg_gt[..., ::-1]
        # Show image + mask of prediction
        img_pred = np.array(img_transposed) * 0.1 + color_seg * 0.9
        img_pred = img_pred.astype(np.uint8)
        # Show image + mask of gt
        img_gt = np.array(img_transposed) * 0.1 + color_seg_gt * 0.9
        img_gt = img_gt.astype(np.uint8)
        # Plot
        axes[i, 0].imshow(img_transposed)
        axes[i, 1].imshow(img_gt)
        axes[i, 2].imshow(img_pred)
        axes[i, 3].imshow(uncertain_map, cmap="jet")
        axes[i, 0].set_title(f"({titles[i]})")
        axes[i, 1].set_title("Ground Truth Mask")
        axes[i, 2].set_title("Predicted Mask")
        axes[i, 3].set_title("Uncertainty Map")
        # im = axes[i, 3].imshow(uncertain_map, cmap="jet")
        # cbar = plt.colorbar(im)
    for ax in axes.flat:
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [5]:
railsem_ds = load_dataset("BhavanaMalla/railsem19-semantic-expanded")
dataset = railsem_ds["data"]

# Define a filtering function
test_list = ['rs02780', 'rs08274', 'rs08482', 'rs03486', 'rs06161', 'rs05167', 'rs08060', 'rs04204', 'rs00481', 'rs05638', 'rs00878', 'rs07079', 'rs03322', 'rs07927', 'rs01719', 'rs07049', 'rs03530', 'rs02408', 'rs01351', 'rs07210']

indices = []
def filter_example(example, idx):
    if example["img_Name"] in test_list:
        indices.append(idx)

# Use the map method to apply the filtering function to each example in the dataset to get the indixes
dataset.map(filter_example, with_indices=True)
test_dataset = dataset.select(indices)
print(indices)
print(test_dataset)

Generating data split:   0%|          | 0/8500 [00:00<?, ? examples/s]

Map:   0%|          | 0/8500 [00:00<?, ? examples/s]

[481, 878, 1351, 1719, 2408, 2780, 3322, 3486, 3530, 4204, 5167, 5638, 6161, 7049, 7079, 7210, 7927, 8060, 8274, 8482]
Dataset({
    features: ['image', 'semantic_mask_label', 'objs_labels', 'objs_bboxes', 'objs_polygons', 'objs_lpolyline', 'objs_polyline-pair', 'img_Height', 'img_Width', 'img_Name'],
    num_rows: 20
})


In [ ]:
# Inference
transforms = get_transforms()
test_ds = CustomDataset(test_dataset, transforms)
test_dataloader = DataLoader(test_ds, batch_size=1, pin_memory=True, shuffle=False, collate_fn=_collate_fn)

#Model and processor
id2label, label2id, labels = get_modified_labels()
model_ckpt = "BhavanaMalla/Mask2Former_Finetuning_cityscapes_swin_railSem19_50epchs_1ptch_1024"
trained_model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, id2label=id2label,
                                                                  label2id=label2id, ignore_mismatched_sizes=True)
processor = AutoImageProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False, do_resize=False,)
processor.num_labels = len(id2label)

test_ece = MulticlassCalibrationError(num_classes=len(id2label), n_bins=15, norm='l1')
test_jaccard_index = JaccardIndex(task="multiclass", num_classes=len(id2label), average='weighted', ignore_index=None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
running_test_iou = []
running_test_ece = []

trained_model.eval()
img_permuted_plt, gt_mask_plt, preds_mask_plt, mean_entropy_plt = [], [], [], []
for step, batch in enumerate(tqdm(test_dataloader)):
    batch_dict = {
        "pixel_values": batch["pixel_values"].to(device),
        "mask_labels": [labels.to(device) for labels in batch["mask_labels"]],
        "class_labels": [labels.to(device) for labels in batch["class_labels"]],
        "pixel_mask": batch["pixel_mask"].to(device),
    }
    with torch.no_grad():
        outputs = trained_model(**batch_dict)
        original_images = batch["original_images"]
        target_sizes = [(img.shape[-2], img.shape[-1]) for img in original_images]
        # post processing
        masks_queries_logits = torch.nn.functional.interpolate(outputs.masks_queries_logits, size=(384, 384), mode="bilinear", align_corners=False)
        class_queries_logits = outputs.class_queries_logits
        masks_classes = class_queries_logits.softmax(dim=-1)[..., :-1]
        masks_probs = masks_queries_logits.sigmoid()
        segmentation = torch.einsum("bqc, bqhw -> bchw", masks_classes, masks_probs) #(bs, C, H, W)


        batch_size = class_queries_logits.shape[0]
        semantic_segmentation = []
        resized = []
        for idx in range(batch_size):
            resized_logits = torch.nn.functional.interpolate(segmentation[idx].unsqueeze(dim=0),
                                                             size=target_sizes[idx], mode="bilinear",
                                                             align_corners=False) #(1, 13, 1080, 1920)

            # resized.append(resized_logits)
            # Compute entropy
            entropy = -torch.sum(resized_logits * torch.log(resized_logits + 1e-9), dim=1)
            mean_entropy = torch.mean(entropy, dim=0)

            semantic_map = resized_logits[0].argmax(dim=0)
            semantic_segmentation.append(semantic_map)

            img_permuted_plt.append(original_images[idx])
            gt_mask_plt.append(batch["original_segmentation_maps"][idx])
            preds_mask_plt.append(semantic_map.unsqueeze(0))
            mean_entropy_plt.append(mean_entropy.unsqueeze(0))
        # Metric
        target = torch.stack(batch["original_segmentation_maps"])
        preds = torch.stack([t.detach() for t in semantic_segmentation])
        # print(segmentation.shape, target.shape, torch.cat(resized).shape)
        # ece_per_cls_batch = test_ece(torch.cat(resized), target.long())
        # running_test_ece.append(test_ece(torch.cat(resized), target.long()).item())
        running_test_iou.append(test_jaccard_index(preds.long(), target.long()).item())
        del outputs, semantic_segmentation, batch_dict

print(f"In Domain MIoU: {statistics.mean(running_test_iou)}")
# print(f"Per Class ECE compute: {test_ece.compute()}")
# print(f"Per Class ECE: {statistics.mean(running_test_ece)}")
titles = ["None"]
for i in range(len(img_permuted_plt)):
    titles.append(f"Image: {i}")
plot_predictions(img_permuted_plt, gt_mask_plt, preds_mask_plt, mean_entropy_plt, titles=titles)

Output hidden; open in https://colab.research.google.com to view.

1h without ece

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import json
from matplotlib import colors
import matplotlib.patches as mpatches

def get_rs_plotting_elements_new_annot():
    class_mapping = {
        0: ["road"], 1: ["sidewalk"], 2: ['construction', 'fence'],
        3: ['rail_raised', 'rail_embedded'], 4: ['pole', 'traffic_light', 'traffic_sign'],
        5: ['sky'], 6: ['human'], 7: ['tram_track', 'rail_track'], 8: ['car', 'truck'],
        9: ['on_rails'], 10: ['vegetation'], 11: ['trackbed'], 12: ['background', 'terrain']
    }
    color_mapping = {
        0: np.array([47, 79, 79]) / 255, 1: np.array([1, 0, 1]),
        2: np.array([160, 82, 45]) / 255, 3: np.array([1, 1, 0]),
        4: np.array([255, 235, 205]) / 255, 5: np.array([0, 1, 1]),
        6: np.array([1, 0, 0]), 7: np.array([70, 130, 180]) / 255,
        8: np.array([176, 48, 96]) / 255, 9: np.array([225, 228, 225]) / 255,
        10: np.array([0, 1, 0]), 11: np.array([47, 79, 79]) / 255,
        12: np.array([0, 0, 0])
    }
    cmap = colors.ListedColormap([color_mapping[i] for i in range(len(class_mapping.keys()))])
    bounds = [i for i in range(0, len(class_mapping.keys()))]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    patches = [
        mpatches.Patch(color=cmap.colors[i], label=str(class_mapping[i]))
        for i in range(len(class_mapping.keys()))
    ]
    return class_mapping, color_mapping, cmap, norm, patches

def get_figs(data, titles=None, is_fpn=False):
    _, _, cmap, norm, patches = get_rs_plotting_elements_new_annot()
    fig, axes = plt.subplots(nrows=len(data), ncols=4, figsize=(25, 5*len(data)))
    axes = np.atleast_2d(axes)
    for ax in axes.flat:
        ax.axis("off")
    for i in data:
        print(i[0].shape, i[1].shape, i[2].shape, i[3].shape)
    # imgs, gt_masks, pred_masks, uc_maps = data
    # for i, (img, gt_mask, pred_mask, uncertain_map) in enumerate(zip(imgs, gt_masks, pred_masks, uc_maps)): #*data
    #     print(img.shape, gt_mask.shape, pred_mask.shape, uncertain_map.shape)
    # for i, (img, gt_mask, pred_mask, uncertain_map) in enumerate(zip(*data)):
        # print(f"Iteration {i}: {img.shape}, {gt_mask.shape}, {pred_mask.shape}, {uncertain_map.shape}")
    for i, imgs in enumerate(data):
        img_transposed = imgs[0].squeeze().permute(1, 2, 0)
        gt_mask = imgs[1].squeeze()
        pred_mask = imgs[2].squeeze()
        uncertain_map = imgs[3].squeeze()

        print(img_transposed.shape, gt_mask.shape, pred_mask.shape, uncertain_map.shape)
        # Plot
        axes[i, 0].imshow(img_transposed)
        axes[i, 0].set_title("Original Image")
        axes[i, 0].set_aspect('auto')
        axes[i, 1].imshow(gt_mask.numpy(), cmap=cmap, norm=norm)
        axes[i, 1].set_title("Ground Truth Mask")
        axes[i, 1].set_aspect('auto')
        axes[i, 2].imshow(pred_mask.numpy(), cmap=cmap, norm=norm)
        axes[i, 2].set_title("Predicted Mask")
        axes[i, 2].set_aspect('auto')
        ucbar = axes[i, 3].imshow(uncertain_map.numpy(), cmap="jet")
        axes[i, 3].set_title("Uncertainty Map")
        axes[i, 3].set_aspect('auto')
        fig.subplots_adjust(right=0.85)
        # axes[i, 1].legend(handles=patches, loc=2, bbox_to_anchor=(1.05, 1), ncols=2)
        fig.legend(handles=patches, loc=2, bbox_to_anchor = (1.05, 1), ncols=2, shadow=True, fontsize="xx-large", borderaxespad=0.)
        fig.colorbar(ucbar, ax=axes[i, 3])
    plt.tight_layout()
    plt.show()
    return fig

In [15]:
# Inference
transforms = get_transforms()
test_ds = CustomDataset(test_dataset, transforms)
test_dataloader = DataLoader(test_ds, batch_size=1, pin_memory=True, shuffle=False, collate_fn=_collate_fn)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Model and processor
id2label, label2id, labels = get_modified_labels()
model_ckpt = "BhavanaMalla/Mask2Former_Finetuning_cityscapes_swin_railSem19_50epchs_1ptch_1024"
trained_model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, id2label=id2label,
                                                                  label2id=label2id, ignore_mismatched_sizes=True)
processor = AutoImageProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False, do_resize=False,)
processor.num_labels = len(id2label)


trained_model.eval()
selected_indices = []
img_permuted_plt, gt_mask_plt, preds_mask_plt, mean_entropy_plt = [], [], [], []
for step, batch in enumerate(tqdm(test_dataloader)):
    if step > 3:
        break
    batch_dict = {
        "pixel_values": batch["pixel_values"].to(device),
        "mask_labels": [labels.to(device) for labels in batch["mask_labels"]],
        "class_labels": [labels.to(device) for labels in batch["class_labels"]],
        "pixel_mask": batch["pixel_mask"].to(device),
    }
    with torch.no_grad():
        outputs = trained_model(**batch_dict)
        original_images = batch["original_images"]
        target_sizes = [(img.shape[-2], img.shape[-1]) for img in original_images]
        # post processing
        masks_queries_logits = torch.nn.functional.interpolate(outputs.masks_queries_logits, size=(384, 384), mode="bilinear", align_corners=False)
        class_queries_logits = outputs.class_queries_logits
        masks_classes = class_queries_logits.softmax(dim=-1)[..., :-1]
        masks_probs = masks_queries_logits.sigmoid()
        segmentation = torch.einsum("bqc, bqhw -> bchw", masks_classes, masks_probs) #(bs, C, H, W)

        batch_size = class_queries_logits.shape[0]
        semantic_segmentation = []
        resized = []
        for idx in range(batch_size):
            resized_logits = torch.nn.functional.interpolate(segmentation[idx].unsqueeze(dim=0),
                                                             size=target_sizes[idx], mode="bilinear",
                                                             align_corners=False) #(1, 13, 1080, 1920)

            # resized.append(resized_logits)
            # Compute entropy
            entropy = -torch.sum(resized_logits * torch.log(resized_logits + 1e-9), dim=1)
            mean_entropy = torch.mean(entropy, dim=0)

            semantic_map = resized_logits[0].argmax(dim=0)
            semantic_segmentation.append(semantic_map)

            img_permuted_plt.append(original_images[idx])
            gt_mask_plt.append(batch["original_segmentation_maps"][idx])
            preds_mask_plt.append(semantic_map.unsqueeze(0))
            mean_entropy_plt.append(mean_entropy.unsqueeze(0))
        # Metric
        target = torch.stack(batch["original_segmentation_maps"])
        preds = torch.stack([t.detach() for t in semantic_segmentation])
        if len(selected_indices) < 5:
            idx = 0
            selected_indices.append((original_images[idx], target[idx], preds[idx], mean_entropy_plt[idx]))
        del outputs, semantic_segmentation, batch_dict



[INFO]: Found labels_info.json.Skipping Download...


  0%|          | 0/20 [00:00<?, ?it/s]

In [20]:
len(selected_indices)

4

In [29]:
for i in selected_indices:
    print(i[0].shape, i[1].shape, i[2].shape, i[3].shape)

torch.Size([3, 1080, 1920]) torch.Size([1080, 1920]) torch.Size([1080, 1920]) torch.Size([1, 1080, 1920])
torch.Size([3, 1080, 1920]) torch.Size([1080, 1920]) torch.Size([1080, 1920]) torch.Size([1, 1080, 1920])
torch.Size([3, 1080, 1920]) torch.Size([1080, 1920]) torch.Size([1080, 1920]) torch.Size([1, 1080, 1920])
torch.Size([3, 1080, 1920]) torch.Size([1080, 1920]) torch.Size([1080, 1920]) torch.Size([1, 1080, 1920])


In [51]:
get_figs(selected_indices, titles=["img", "gt", "pred", "uncertainty"])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def sample_logits_and_labels(logits, true_masks, num_pixels=20000):

    bs, classes, H, W = logits.shape
    logits_flat = logits.view(bs, classes, -1)  # Flatten spatial dimensions for logits
    true_masks_flat = true_masks.view(bs, -1)  # Flatten spatial dimensions for true masks
    # print(logits_flat.shape, true_masks_flat.shape)
    indices = torch.randint(0, H * W, (bs, num_pixels))  # Randomly sample indices
    # print(indices.shape)

    sampled_logits = torch.gather(logits_flat, 2, indices.unsqueeze(1).expand(-1, classes, -1))  # Gather logits based on sampled indices
    sampled_logits = sampled_logits.view(bs, classes, num_pixels)  # Reshape back to (bs, classes, num_pixels)

    sampled_labels = torch.gather(true_masks_flat.unsqueeze(1), 2, indices.unsqueeze(1))  # Gather true labels based on sampled indices
    sampled_labels = sampled_labels.squeeze(1)  # Remove the extra dimension

    return sampled_logits, sampled_labels

In [ ]:
# Inference
transforms = get_transforms()
test_ds = CustomDataset(test_dataset, transforms)
test_dataloader = DataLoader(test_ds, batch_size=1, pin_memory=True, shuffle=False, collate_fn=_collate_fn)

#Model and processor
id2label, label2id, labels = get_modified_labels()
model_ckpt = "BhavanaMalla/Mask2Former_Finetuning_cityscapes_swin_railSem19_50epchs_1ptch_1024"
trained_model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, id2label=id2label,
                                                                  label2id=label2id, ignore_mismatched_sizes=True)
processor = AutoImageProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False, do_resize=False,)
processor.num_labels = len(id2label)

test_ece = MulticlassCalibrationError(num_classes=len(id2label), n_bins=15, norm='l1')
test_jaccard_index = JaccardIndex(task="multiclass", num_classes=len(id2label), average='weighted', ignore_index=None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
running_test_iou = []
running_test_ece = []

trained_model.eval()
img_permuted_plt, gt_mask_plt, preds_mask_plt, mean_entropy_plt = [], [], [], []

for step, batch in enumerate(tqdm(test_dataloader)):
    batch_dict = {
        "pixel_values": batch["pixel_values"].to(device),
        "mask_labels": [labels.to(device) for labels in batch["mask_labels"]],
        "class_labels": [labels.to(device) for labels in batch["class_labels"]],
        "pixel_mask": batch["pixel_mask"].to(device),
    }
    with torch.no_grad():
        outputs = trained_model(**batch_dict)
        original_images = batch["original_images"]
        target_sizes = [(img.shape[-2], img.shape[-1]) for img in original_images]
        # post processing
        masks_queries_logits = torch.nn.functional.interpolate(outputs.masks_queries_logits, size=(384, 384), mode="bilinear", align_corners=False)
        class_queries_logits = outputs.class_queries_logits
        masks_classes = class_queries_logits.softmax(dim=-1)[..., :-1]
        masks_probs = masks_queries_logits.sigmoid()
        segmentation = torch.einsum("bqc, bqhw -> bchw", masks_classes, masks_probs) #(bs, C, H, W)


        batch_size = class_queries_logits.shape[0]
        semantic_segmentation = []
        resized = []
        for idx in range(batch_size):
            resized_logits = torch.nn.functional.interpolate(segmentation[idx].unsqueeze(dim=0),
                                                             size=target_sizes[idx], mode="bilinear",
                                                             align_corners=False) #(1, 13, 1080, 1920)

            resized.append(resized_logits)
            # Compute entropy
            entropy = -torch.sum(resized_logits * torch.log(resized_logits + 1e-9), dim=1) #1, 1080, 1920
            # mean_entropy = torch.mean(entropy, dim=0) #1080, 1920 #i think not needed..we just need to squeeze the entropy i guess
            mean_entropy = entropy.squeeze()
            semantic_map = resized_logits[0].argmax(dim=0)
            semantic_segmentation.append(semantic_map)

            img_permuted_plt.append(original_images[idx])
            gt_mask_plt.append(batch["original_segmentation_maps"][idx])
            preds_mask_plt.append(semantic_map.unsqueeze(0))
            mean_entropy_plt.append(mean_entropy.unsqueeze(0))
        # Metric
        target = torch.stack(batch["original_segmentation_maps"])
        preds = torch.stack([t.detach() for t in semantic_segmentation])
        resized_logits = torch.cat(resized)
        # print(segmentation.shape, target.shape, resized_logits.shape)
        # sampled_logits, sampled_labels = sample_logits_and_labels(resized_logits, target)
        # print(sampled_logits.shape, sampled_labels.shape)
        ece_per_cls_batch = test_ece(resized_logits, target)
        running_test_ece.append(test_ece(resized_logits, target.long()).item())
        running_test_iou.append(test_jaccard_index(preds.long(), target.long()).item())
        del outputs, semantic_segmentation, batch_dict

print(f"In Domain MIoU: {statistics.mean(running_test_iou)}")
print(f"Per Class ECE compute: {test_ece.compute()}")
print(f"Per Class ECE: {statistics.mean(running_test_ece)}")

titles = ["None"]
for i in range(len(img_permuted_plt)):
    titles.append(f"Image: {i}")
plot_predictions(img_permuted_plt, gt_mask_plt, preds_mask_plt, mean_entropy_plt, titles=titles)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
running_test_ece

[0.7477465271949768,
 0.6835840940475464,
 0.6620702743530273,
 0.33867010474205017,
 0.7205995321273804,
 0.7203750014305115,
 0.7389358878135681,
 0.7561261653900146,
 0.733186662197113,
 0.7230852842330933,
 0.7151497006416321,
 0.7521498203277588,
 0.7135846018791199,
 0.6903244256973267,
 0.7366073727607727,
 0.6911730766296387,
 0.6862770318984985,
 0.7221536040306091,
 0.7251657247543335,
 0.7097748517990112]

In [ ]:
len(resized), target.shape, resized_logits.shape, torch.cat([resized_logits]).shape, torch.stack(batch["original_segmentation_maps"]).shape

(1,
 torch.Size([1, 1080, 1920]),
 torch.Size([1, 13, 1080, 1920]),
 torch.Size([1, 13, 1080, 1920]),
 torch.Size([1, 1080, 1920]))

In [ ]:
import torch

test_ece = MulticlassCalibrationError(num_classes=10, n_bins=15, norm='l1')

# Example logits tensor shape: (bs, classes, H, W)
logits = torch.randn(4, 10, 256, 256)  # Example shape, replace with actual logits tensor
true_masks = torch.randint(0, 10, (4, 256, 256))

def sample_logits_and_labels(logits, true_masks, num_pixels=20000):

    bs, classes, H, W = logits.shape
    logits_flat = logits.view(bs, classes, -1)  # Flatten spatial dimensions for logits
    true_masks_flat = true_masks.view(bs, -1)  # Flatten spatial dimensions for true masks
    print(logits_flat.shape)
    print(true_masks_flat)
    indices = torch.randint(0, H * W, (bs, num_pixels))  # Randomly sample indices
    print(indices.shape)

    sampled_logits = torch.gather(logits_flat, 2, indices.unsqueeze(1).expand(-1, classes, -1))  # Gather logits based on sampled indices
    sampled_logits = sampled_logits.view(bs, classes, num_pixels)  # Reshape back to (bs, classes, num_pixels)

    sampled_labels = torch.gather(true_masks_flat.unsqueeze(1), 2, indices.unsqueeze(1))  # Gather true labels based on sampled indices
    sampled_labels = sampled_labels.squeeze(1)  # Remove the extra dimension

    return sampled_logits, sampled_labels

# Sample 20k pixels from each image along with their corresponding true labels
sampled_logits, sampled_labels = sample_logits_and_labels(logits, true_masks)

print("Sampled logits shape:", sampled_logits.shape)
print("Sampled labels shape:", sampled_labels.shape)

print(test_ece(logits, true_masks))
print(test_ece(sampled_logits, sampled_labels))


torch.Size([4, 10, 65536])
tensor([[2, 7, 4,  ..., 6, 9, 1],
        [2, 8, 8,  ..., 7, 4, 4],
        [0, 7, 4,  ..., 1, 7, 1],
        [6, 8, 6,  ..., 8, 0, 3]])
torch.Size([4, 20000])
Sampled logits shape: torch.Size([4, 10, 20000])
Sampled labels shape: torch.Size([4, 20000])
tensor(0.2198)
tensor(0.2202)


In [ ]:
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt


transform = [None]
transform.extend([A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=True) for _ in range(9)])
nrows = len(transform)
ncols = 4

img_permuted_plt, gt_mask_plt, preds_mask_plt, mean_entropy_plt = [], [], [], []
for row in range(nrows):
    if transform[row] is None:
        img_permuted = image
        if img_permuted.ndim == 3:
            img_permuted = img_permuted.unsqueeze(0)
    else:
        sm = image.squeeze().numpy().transpose(1, 2, 0)
        augmented_img = transform[row](image=sm)['image']
        img_permuted = torch.from_numpy(augmented_img).permute(2, 0, 1).unsqueeze(0)
    print(img_permuted.shape)

    semantic_inputs = processor(images=img_permuted, task_inputs=["semantic"], return_tensors="pt")
    trained_model.eval()
    trained_model.model.is_training = False
    with torch.no_grad():
        outputs = trained_model(**semantic_inputs)
        target_sizes=[(img_permuted.shape[-2], img_permuted.shape[-1])]
        masks_classes = outputs.class_queries_logits.softmax(dim=-1)[..., :-1]
        masks_probs = outputs.masks_queries_logits.sigmoid()
        segmentation = torch.einsum("bqc, bqhw -> bchw", masks_classes, masks_probs)
        resized_logits = torch.nn.functional.interpolate(segmentation, size=target_sizes[0], mode="bilinear", align_corners=False)
        semantic_map = resized_logits[0].argmax(dim=0)

        # Compute entropy
        entropy = -torch.sum(resized_logits * torch.log(resized_logits + 1e-9), dim=1)
        mean_entropy = torch.mean(entropy, dim=0)

        img_permuted_plt.append(img_permuted)
        gt_mask_plt.append(gt_mask)
        preds_mask_plt.append(semantic_map.unsqueeze(0))
        mean_entropy_plt.append(mean_entropy.unsqueeze(0))
        del outputs